In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [ ]:
# Problem parameters
sigma_s = 0 # Macroscopic cross section for scattering [cm^(-1)] {Play with sigma_s = ln(2) or something}
sigma_t = 1 # Macroscopic total interaction cross section [cm^(-1)] (reciprocal is neutron mean free path (mfp))
q_ext = 3 # External source density rate [TODO: insert units]
q = q_ext/2.0 # Source density rate [TODO: insert units]

# Left- and right-endpoints of domain
a = 0
b = 10

# Number of cells in domain
ncells = 9
# Number of points in domain
npts   = ncells + 1
points = np.linspace(a, b, npts)
dx     = np.abs(points[1] - points[0])

# Auxillary term for calculation clarity in the loops below
aux_term = (sigma_t/2.0)*dx

# BCs
psi_left = 1.5
psi_right = 1.5

# Quadrature order (needs to be EVEN to avoid mu = 0)
m = 16
angles, weights = scipy.special.roots_legendre(m)
# By default, angles go from negative to positive, and we want the opposite
angles = np.flip(angles)

# Array to store angular fluxes in all directions at the previous cell interface
psi_prev_i = np.zeros((m, ))

# Enforce BCs
psi_prev_i[0:(m/2)] = psi_left
psi_prev_i[(m/2):]  = psi_right

# Array to store scalar fluxes at the midpoint of each cell
scalar_fluxes_mid = np.zeros((ncells, ))

# If the absolute difference of successive iterates of the scalar flux at
# cell interface i is below this, then we stop updating the scalar flux at 
# that interface
eps = 1e-14

for i in np.arange(1, npts):
    scalar_fluxes_mid[i] = np.log(2) # Initial guess for the midpoint scalar flux in cell i
    next_sca_flux_mid_i = 0.0

    sca_flux_mid_i_diff = np.inf
    
    # Initial guess for the source term at midpoint of cell i
    curr_source_mid_i = (q_ext/2.0) + (sigma_s/2.0)*scalar_fluxes_mid[i]

    while sca_flux_mid_i_diff >= eps: 
        for j, mu_j in enumerate(angles):
            abs_mu_j = np.abs(mu_j) 
            psi_next_i_j = ((abs_mu_j - aux_term)*psi_prev_i[j] + curr_source_mid_i*dx) / \
                            (abs_mu_j + aux_term)

            # Update scalar flux at cell interface i using the average of the
            # angular flux at the previous cell interace and the just computed angular
            # flux at the current cell interface 
            psi_avg_i_j            = (psi_next_i_j + psi_prev_i[j])/2.0
            next_sca_flux_mid_i   += psi_avg_i_j * weights[j]

            # If the difference is small enough, set the angular flux at the 
            # previous cell interface to be the angular flux at the current interface
            if sca_flux_mid_i_diff < eps:
                psi_prev_i[j] = psi_next_i_j

        sca_flux_mid_i_diff = np.abs(next_sca_flux_mid_i - scalar_fluxes_mid[i])

        # Update the source term
        scalar_fluxes_mid[i] = next_sca_flux_mid_i
        curr_source_mid_i    = (q_ext/2.0) + (sigma_s/2.0)*scalar_fluxes_mid[i]
        next_sca_flux_mid_i  = 0.0